In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
from torch.utils.data import Dataset, DataLoader

In [2]:
#based on https://dzlab.github.io/timeseries/2018/11/25/LSTM-FCN-pytorch-part-1/

class BlockFCNConv(nn.Module):
    def __init__(self, in_channel=1, out_channel=128, kernel_size=8, momentum=0.99, epsilon=0.001, squeeze=False):
        super().__init__()
        self.conv = nn.Conv1d(in_channel, out_channel, kernel_size=kernel_size)
        self.batch_norm = nn.BatchNorm1d(num_features=out_channel, eps=epsilon, momentum=momentum)
        self.relu = nn.ReLU()
    def forward(self, x):
        # input (batch_size, num_variables, time_steps), e.g. (128, 1, 512)
        x = self.conv(x)
        # input (batch_size, out_channel, L_out)
        x = self.batch_norm(x)
        # same shape as input
        y = self.relu(x)
        return y
    
class FCN(nn.Module):
    def __init__(self, time_steps, channels=[1, 128, 256, 128], kernels=[8, 5, 3], output_dim =1, mom=0.99, eps=0.001):
        super().__init__()
        self.conv1 = BlockFCNConv(channels[0], channels[1], kernels[0], momentum=mom, epsilon=eps, squeeze=True)
        self.conv2 = BlockFCNConv(channels[1], channels[2], kernels[1], momentum=mom, epsilon=eps, squeeze=True)
        self.conv3 = BlockFCNConv(channels[2], channels[3], kernels[2], momentum=mom, epsilon=eps)
        output_size = time_steps - sum(kernels) + len(kernels)
        self.global_pooling = nn.AvgPool1d(kernel_size=output_size)
        self.linear = nn.Linear(channels[2], output_dim )
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        # apply Global Average Pooling 1D
        x = self.global_pooling(x)
        x = self.linear(x.squeeze())
        return y

In [3]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

model = FCN(13, channels=[13, 128, 128, 128])
count_parameters(model)

145665

In [4]:
for name, params in model.named_parameters():
    print(name)

conv1.conv.weight
conv1.conv.bias
conv1.batch_norm.weight
conv1.batch_norm.bias
conv2.conv.weight
conv2.conv.bias
conv2.batch_norm.weight
conv2.batch_norm.bias
conv3.conv.weight
conv3.conv.bias
conv3.batch_norm.weight
conv3.batch_norm.bias
linear.weight
linear.bias


In [5]:

class LSTMModel(nn.Module):
    
    def __init__(self, batch_size, seq_len, input_dim, n_layers, hidden_dim, output_dim, lin_hidden_dim = 50):
        super(LSTMModel, self).__init__()

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)#
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.n_layers = n_layers
        self.hidden = self.init_hidden()
        self.input_dim = input_dim
        
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.n_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.n_layers, self.batch_size, self.hidden_dim))
        
    def forward(self, x):
        
        output, (hn, cn) = self.lstm(x)
        out1 = self.linear(hn[-1].view(len(x),-1))

        return out1

In [6]:
model = LSTMModel(64, 15, 13, 2, 120, 1)
count_parameters(model)

181081

In [13]:
import pickle
from ts_dataset import TSDataset


dataset_name = "POLLUTION"
if dataset_name == "POLLUTION":
    task_size = 50
    window_size = 5
    input_dim = 14

elif dataset_name == "HR":
    task_size = 50
    window_size = 32
    input_dim = 13 

elif dataset_name == "BATTERY":
    task_size = 50
    window_size = 20
    input_dim = 3

kernels = [8,5,3] if dataset_name!= "POLLUTION" else [4,2,1]

In [14]:
train_data = pickle.load(  open( "../Data/TRAIN-"+dataset_name+"-W"+str(window_size)+"-T"+str(task_size)+"-NOML.pickle", "rb" ) )


In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

x = np.transpose(train_data.x[:64], [0,2,1])
y = train_data.y[:64]
        
x = torch.tensor(x).float().to(device)
y = torch.tensor(y).float().to(device)

In [16]:

model = FCN(time_steps = window_size,  channels=[input_dim, 128, 128, 128] , kernels=kernels)
model.cuda()

FCN(
  (conv1): BlockFCNConv(
    (conv): Conv1d(14, 128, kernel_size=(4,), stride=(1,))
    (batch_norm): BatchNorm1d(128, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2): BlockFCNConv(
    (conv): Conv1d(128, 128, kernel_size=(2,), stride=(1,))
    (batch_norm): BatchNorm1d(128, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv3): BlockFCNConv(
    (conv): Conv1d(128, 128, kernel_size=(1,), stride=(1,))
    (batch_norm): BatchNorm1d(128, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (global_pooling): AvgPool1d(kernel_size=(1,), stride=(1,), padding=(0,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [17]:
y_pred = model(x)

In [18]:
y_pred.shape

torch.Size([64, 1])

In [22]:

child = list(model.children())

In [23]:
child[-1]

Linear(in_features=128, out_features=1, bias=True)

In [28]:
seq = nn.Sequential(model)
seq(x).shape

torch.Size([64, 1])

In [34]:
for layer in seq.children:
    
    print(layer(x).shape)

TypeError: 'method' object is not iterable

In [40]:
model.linear.in_features

128

RuntimeError: size mismatch, m1: [896 x 5], m2: [128 x 1] at C:/w/1/s/tmp_conda_3.7_100118/conda/conda-bld/pytorch_1579082551706/work/aten/src\THC/generic/THCTensorMathBlas.cu:290